This is a local, non-Kaggle notebook in which TFX 1.16.0 and python 3.10 and the compatible versions of other libraries are installed in a virtual environment that this notebook is running in.

paths are relative to the github repository directory, "recommender_systems"

In [1]:
import shutil

from tfx.orchestration import metadata

import tensorflow_transform as tft

from ml_metadata.proto import metadata_store_pb2
from ml_metadata.metadata_store import metadata_store

import sys
import os
sys.path.append(os.path.join(os.getcwd(), "src/test/python/movie_lens_tfx"))
sys.path.append(os.path.join(os.getcwd(), "src/main/python/movie_lens_tfx"))

from helper import *
from movie_lens_tfx.PipelineComponentsFactory import *
from movie_lens_tfx.tune_train_movie_lens import *

from absl import logging
tf.get_logger().propagate = False
logging.set_verbosity(logging.WARNING)
logging.set_stderrthreshold(logging.WARNING)

2025-11-06 11:48:37.199068: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-06 11:48:37.202511: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-06 11:48:37.250542: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-06 11:48:38.157703: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
INFO:absl:tensorflow_io is not available: No module named 'tensorflow_io'
INFO:absl:tensorflow_ranking is not available: No module named 'tensorflow_ranking'
INFO:absl:tensorflow_text is not available: No module named 'tensorflow_text'
INFO:absl:tensorflow_decision_forests is

## EDA on the raw data

### w/ Polars and Plotly express
output is written to bin/local_notebook/images/

In [2]:
%run src/main/python/eda/eda_raw.py

CWD=/home/nichole/projects/github/recommender_systems, kaggle=False
177 movies were not rated
0 users did not rate

ratings: HEAD
shape: (5, 4)
┌─────────┬──────────┬────────┬───────────┐
│ user_id ┆ movie_id ┆ rating ┆ timestamp │
│ ---     ┆ ---      ┆ ---    ┆ ---       │
│ i64     ┆ i64      ┆ i64    ┆ i64       │
╞═════════╪══════════╪════════╪═══════════╡
│ 1       ┆ 1193     ┆ 5      ┆ 978300760 │
│ 1       ┆ 661      ┆ 3      ┆ 978302109 │
│ 1       ┆ 914      ┆ 3      ┆ 978301968 │
│ 1       ┆ 3408     ┆ 4      ┆ 978300275 │
│ 1       ┆ 2355     ┆ 5      ┆ 978824291 │
└─────────┴──────────┴────────┴───────────┘

ratings: DESCRIBE
<bound method DataFrame.describe of shape: (1_000_209, 4)
┌─────────┬──────────┬────────┬───────────┐
│ user_id ┆ movie_id ┆ rating ┆ timestamp │
│ ---     ┆ ---      ┆ ---    ┆ ---       │
│ i64     ┆ i64      ┆ i64    ┆ i64       │
╞═════════╪══════════╪════════╪═══════════╡
│ 1       ┆ 1193     ┆ 5      ┆ 978300760 │
│ 1       ┆ 661      ┆ 3      ┆

the generated images aren't plotted here, but similar plots are shown via TFDV below.

### Run data pre-processing on full dataset to get the transformed data

In [3]:
infiles_dict_ser, output_config_ser, split_names = get_test_data(use_small=False)
user_id_max = 6040
movie_id_max = 3952
n_genres = N_GENRES
n_age_groups = N_AGE_GROUPS
n_occupations = 21
MIN_EVAL_SIZE = 50 #make this larger for production pipeline

test_num = "1"
    
PIPELINE_NAME = 'TestPipelines'
output_data_dir = os.path.join(get_bin_dir(), "local_notebook", test_num)
PIPELINE_ROOT = os.path.join(output_data_dir, PIPELINE_NAME)

# remove results from previous test runs:
try:
  print(f"removing: {PIPELINE_ROOT}")
  shutil.rmtree(PIPELINE_ROOT)
except OSError as e:
  pass
METADATA_PATH = os.path.join(PIPELINE_ROOT, 'tfx_metadata',
                             'metadata.db')
os.makedirs(os.path.join(PIPELINE_ROOT, 'tfx_metadata'),
            exist_ok=True)

ENABLE_CACHE = True

# metadata_connection_config = metadata_store_pb2.ConnectionConfig()
# metadata_connection_config.sqlite.SetInParent()
# metadata_connection = metadata.Metadata(metadata_connection_config)
metadata_connection_config = metadata.sqlite_metadata_connection_config(
  METADATA_PATH)

store = metadata_store.MetadataStore(metadata_connection_config)

if get_kaggle():
  tr_dir = "/kaggle/working/"
else:
  tr_dir = os.path.join(get_project_dir(), "src/main/python/movie_lens_tfx")

serving_model_dir = os.path.join(PIPELINE_ROOT, 'serving_model')
output_parquet_path = os.path.join(PIPELINE_ROOT, "transformed_parquet")

# for the custom ingestion component, the apache beam pipeline needs to be able to
# find the sibling scripts it imports.
# 2 solutions: (1) create a tar archive and use --extra_package in pipeline args
# or (2) use setup.py and --setup_file in pipeline args.

beam_pipeline_args = [
  '--direct_running_mode=multi_processing',
  '--direct_num_workers=0',
  '--setup_file=setup.py',
  #f'--extra_package={ingest_tar_file}'
]

CWD=/home/nichole/projects/github/recommender_systems, kaggle=False
removing: /home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines


In [4]:
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

tf.get_logger().propagate = False
import logging
logging.getLogger().setLevel(logging.WARNING)
from absl import logging
logging.set_verbosity(logging.WARNING)
logging.set_stderrthreshold(logging.WARNING)

context = InteractiveContext(pipeline_name=PIPELINE_NAME, pipeline_root=PIPELINE_ROOT,
  metadata_connection_config=metadata_connection_config,
  beam_pipeline_args=beam_pipeline_args
)

factory = PipelineComponentsFactory(infiles_dict_ser, output_config_ser, tr_dir,
    user_id_max, movie_id_max, n_genres, n_age_groups,
    MIN_EVAL_SIZE, serving_model_dir, output_parquet_path)

components = factory.build_components(PIPELINE_TYPE.PREPROCESSING)

for component in components:
    context.run(component)

print(f'done pre-processing data')

2025-11-06 11:49:47.479711: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-06 11:49:47.487605: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-06 11:49:47.633525: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-06 11:49:47.653692: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-06 11:49:47.660001: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-06 11:49:47.760063: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optim

running bdist_wheel
running build
running build_py
creating build/lib
copying transform_movie_lens.py -> build/lib
copying PipelineComponentsFactory.py -> build/lib
copying tune_train_movie_lens.py -> build/lib
installing to /tmp/tmpioct242_
running install
running install_lib
copying build/lib/transform_movie_lens.py -> /tmp/tmpioct242_/.
copying build/lib/PipelineComponentsFactory.py -> /tmp/tmpioct242_/.
copying build/lib/tune_train_movie_lens.py -> /tmp/tmpioct242_/.
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Trans

/home/nichole/miniconda3/envs/tfx_py310/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:90: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        This deprecation is overdue, please update your project and remove deprecated
        calls to avoid build errors in the future.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


Processing ./bin/local_notebook/1/TestPipelines/_wheels/tfx_user_code_transform-0.0+67ad0c369a791fa06bd31d53570fab633abc1151f7552cf29ccd1825c20e375e-py3-none-any.whl


INFO:absl:udf_utils.get_fn {'module_file': None, 'module_path': 'transform_movie_lens@/home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines/_wheels/tfx_user_code_transform-0.0+67ad0c369a791fa06bd31d53570fab633abc1151f7552cf29ccd1825c20e375e-py3-none-any.whl', 'stats_options_updater_fn': None} 'stats_options_updater_fn'
INFO:absl:Installing '/home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines/_wheels/tfx_user_code_transform-0.0+67ad0c369a791fa06bd31d53570fab633abc1151f7552cf29ccd1825c20e375e-py3-none-any.whl' to a temporary directory.
INFO:absl:Executing: ['/home/nichole/miniconda3/envs/tfx_py310/bin/python3.10', '-m', 'pip', 'install', '--target', '/tmp/tmp1gewpgbl', '/home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines/_wheels/tfx_user_code_transform-0.0+67ad0c369a791fa06bd31d53570fab633abc1151f7552cf29ccd1825c20e375e-py3-none-any.whl']


...

INFO:absl:Successfully installed '/home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines/_wheels/tfx_user_code_transform-0.0+67ad0c369a791fa06bd31d53570fab633abc1151f7552cf29ccd1825c20e375e-py3-none-any.whl'.
INFO:absl:Installing '/home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines/_wheels/tfx_user_code_transform-0.0+67ad0c369a791fa06bd31d53570fab633abc1151f7552cf29ccd1825c20e375e-py3-none-any.whl' to a temporary directory.
INFO:absl:Executing: ['/home/nichole/miniconda3/envs/tfx_py310/bin/python3.10', '-m', 'pip', 'install', '--target', '/tmp/tmp0h8y59_t', '/home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines/_wheels/tfx_user_code_transform-0.0+67ad0c369a791fa06bd31d53570fab633abc1151f7552cf29ccd1825c20e375e-py3-none-any.whl']


Processing ./bin/local_notebook/1/TestPipelines/_wheels/tfx_user_code_transform-0.0+67ad0c369a791fa06bd31d53570fab633abc1151f7552cf29ccd1825c20e375e-py3-none-any.whl


INFO:absl:Successfully installed '/home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines/_wheels/tfx_user_code_transform-0.0+67ad0c369a791fa06bd31d53570fab633abc1151f7552cf29ccd1825c20e375e-py3-none-any.whl'.
INFO:absl:Feature genres has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature age has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature gender has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature movie_id has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature occupation has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature rating has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature timestamp has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature user_id has a shape dim {
  size: 1
}
. Setting to DenseTensor.


INFO:absl:Feature genres has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature age has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature gender has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature movie_id has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature occupation has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature rating has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature timestamp has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature user_id has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature genres has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature age has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature gender has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature movie_id has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature occu

done pre-processing data


## EDA on the transformed data

### using Polars, Plotly.express 

this can take an hour on a single COTS

In [5]:
%run src/main/python/eda/eda_transformed.py

CWD=/home/nichole/projects/github/recommender_systems, kaggle=False
CWD=/home/nichole/projects/github/recommender_systems, kaggle=False
wrote multivariate EDA images to /home/nichole/projects/github/recommender_systems/bin/local_notebook/images


Some of the images are shown here from a former saved run.

You can see a clear correlation between rating and movie_id, then less
so between rating and age and rating and occupation.

<img src="src/test/resources/train_dist_corr_heatmap.png">

You can see many cooccurences between movie genres.

<img src="src/test/resources/train_genre_cooccurence_rating_5_heatmap.png">

You can see that Drama, then Comedy, then Action are the most popular movie genres.

<img src="src/test/resources/train_genres_rating_5.png">


### using Pandas, Pyspark MLLIB FPGrowth

This does a market basket analysis with movie_ids.

If you want the PrefixSpan plots also, set
PLOT_PREFIXSPAN = True
but beware that the script will take much longer to run.

In [6]:
PLOT_PREFIXSPAN=False

In [17]:
%run src/main/python/eda/eda_transformed_pyspark_mllib.py

CWD=/home/nichole/projects/github/recommender_systems, kaggle=False
CWD=/home/nichole/projects/github/recommender_systems, kaggle=False
start spark session for 1 local cores


/home/nichole/miniconda3/envs/tfx_py310/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.



root
 |-- movie_ids: array (nullable = true)
 |    |-- element: integer (containsNull = false)

construct FPGrowth model



frequent_itemsets_spark:


+------------------+----+
|             items|freq|
+------------------+----+
|            [2706]| 277|
|      [2706, 3421]|  34|
|      [2706, 3671]|  32|
|      [2706, 2355]|  31|
|      [2706, 1193]|  35|
|      [2706, 2987]|  32|
|       [2706, 223]|  62|
| [2706, 223, 2571]|  31|
| [2706, 223, 2858]|  33|
|       [2706, 480]|  37|
|         [2706, 1]|  43|
|      [2706, 1089]|  35|
|      [2706, 1036]|  46|
|      [2706, 3578]|  61|
|[2706, 3578, 2571]|  31|
|       [2706, 457]|  39|
|      [2706, 2028]|  71|
| [2706, 2028, 318]|  34|
|[2706, 2028, 2571]|  35|
| [2706, 2028, 260]|  32|
+------------------+----+
only showing top 20 rows


filtered_itemsets:


...

root
 |-- movie_ids: array (nullable = true)
 |    |-- element: integer (containsNull = false)

construct FPGrowth model

frequent_itemsets_spark:
+------+----+
| items|freq|
+------+----+
|[2700]|  30|
|[2023]|  29|
|[1653]|  25|
|[3101]|  25|
|[3809]|  33|
|[1608]|  32|
|[2108]|  26|
|[3704]|  28|
|[3450]|  26|
|[2144]|  27|
|[2723]|  27|
|[2797]|  34|
| [353]|  29|
|[2605]|  28|
|[3070]|  24|
| [454]|  32|
|[3263]|  31|
| [316]|  30|
|[1792]|  24|
|[2001]|  44|
+------+----+
only showing top 20 rows


filtered_itemsets:
+-----+----+
|items|freq|
+-----+----+
+-----+----+

results_pd len=0
sorted by support, head(5)=Empty DataFrame
Columns: [items, freq, itemset_str, support]
Index: []


/home/nichole/miniconda3/envs/tfx_py310/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.



root
 |-- movie_ids: array (nullable = true)
 |    |-- element: integer (containsNull = false)

construct FPGrowth model

frequent_itemsets_spark:
+------+----+
| items|freq|
+------+----+
|[1320]|  19|
|[2683]|  18|
|[2746]|  18|
|[1265]|  17|
|[2805]|  17|
|[3697]|  17|
|[1377]|  22|
|[2054]|  20|
| [736]|  18|
| [435]|  18|
|  [70]|  19|
|[1562]|  22|
|[2012]|  17|
| [196]|  17|
|[3175]|  19|
| [485]|  18|
| [185]|  20|
|[2701]|  32|
|[3623]|  26|
|[2722]|  21|
+------+----+
only showing top 20 rows


filtered_itemsets:
+-----+----+
|items|freq|
+-----+----+
+-----+----+

results_pd len=0
sorted by support, head(5)=Empty DataFrame
Columns: [items, freq, itemset_str, support]
Index: []


/home/nichole/miniconda3/envs/tfx_py310/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.



root
 |-- movie_ids: array (nullable = true)
 |    |-- element: integer (containsNull = false)

construct FPGrowth model

frequent_itemsets_spark:
+------+----+
| items|freq|
+------+----+
|[2642]|  15|
| [480]|  11|
| [160]|  13|
|[2422]|  13|
|[2723]|  12|
|[2907]|  11|
|[1976]|  11|
| [737]|  13|
|[2555]|  13|
|[2628]|  19|
|[1917]|  16|
|[3268]|  15|
| [153]|  18|
|[1973]|  11|
|[3785]|  14|
|[1591]|  12|
|[1676]|  13|
| [849]|  15|
|[1381]|  25|
|[3354]|  22|
+------+----+
only showing top 20 rows


filtered_itemsets:
+-----+----+
|items|freq|
+-----+----+
+-----+----+

results_pd len=0
sorted by support, head(5)=Empty DataFrame
Columns: [items, freq, itemset_str, support]
Index: []
wrote multivariate EDA images to /home/nichole/projects/github/recommender_systems/bin/local_notebook/images


Some of the images are shown here from a former saved run.

You can see that many association rules can be derived from the data.
<img src="src/test/resources/train_movies_assoc_rules_rating_5.png">

and that there are many frequent itemsets.
<img src="src/test/resources/train_movies_itemsets_rating_5.png">

<img src="src/test/resources/train_movies_itemsets_rating_5_2.png">

The results show that neural network models will have good material to exploit.

The PrefixScan model can be enabled in the script to provide images to explore sequences
for actions or as precursor to Sequential DNN models.

### Data and Concept Drift
After exploring the data inputs to the model, we want to define monitoring
for data and concept shifts.

let X = features

let Y = targets

Data shift is a change in the joint distribution, P(X, Y). 

Using the probability product rule, we can explore 4 causes for the
simplest changes in P(X,Y)
$$ P(X, Y) = P(Y, X) = P(X|Y)P(Y) = P(Y|X)P(X) $$

We can look for changes in one member in the following pairs at any time
(one is simpler than exploring more than 1 member changing at same time):
$$ P(X|Y) * P(Y) $$
$$ P(Y|X) * P(X) $$

* Covariate shift:
  $$ P(X) changed.  P(Y|X) unchanged $$
  Distr of model inputs changes.
* Label shift:
  $$ P(Y) changed,  P(X|Y) unchanged $$
  Distr of model outputs changes, but for any given output, the input distribution stays the same.
  
* Concept shift:
  $$ P(X) unchanged,  P(Y|X) changed $$
* Manifestation shift:
  $$ P(Y) unchanged,  P(X|Y) changed $$

[see more at NannyML](https://www.nannyml.com/blog/concept-drift)

if the train, eval, and test split are random, then one could use
the maximum differences in their distributions as a lower limit on 
the estimate for stochastic error.  A trigger for data drift should
then be GEQ about 3 times that stochastic error.

The previous data doesn't exist yet for this project, but one could
either download another movie-lens dataset of different time period,
or split this 1M dataset by a timestamp ordering into 2 partitions.

More data is available at [GroupLens](https://files.grouplens.org/datasets/movielens/)

### Data [Drift and Skew using TFDV](https://www.tensorflow.org/tfx/tutorials/data_validation/tfdv_basic)
* Drift
  Drift detection is supported for categorical features and between consecutive spans of data
  (i.e., between span N and span N+1), such as between different days of training data.
    * L-infinity distance
    * alerts when drift is higher than threshold distance
* Skew
    * data - schema skew
      occurs when the training and serving data do not conform to the same schema.
      Any expected deviations between the two (such as the label feature being only present in the training data but not in serving) should be specified through environments field in the schema.
    * feature skew
      occurs when the training and serving feature values are different
      This can happen when:
        * A data source of features is modified between training and serving time
        * A difference in logic for generating between training and serving.
    * distribution skew
      occurs when the training and serving distributions are significantly different.
      some key causes:
        * different code or different data sources to generate the training dataset
        * a faulty sampling mechanism that chooses a non-representative subsample of the serving data to train on.


### using TFDV to look at the raw data

In [14]:
from tfx.dsl.io import fileio
from tfx.orchestration import metadata
from tfx.components import StatisticsGen, SchemaGen, ExampleValidator
from tfx.utils import io_utils
from tensorflow_metadata.proto.v0 import anomalies_pb2, schema_pb2
from tensorflow_metadata.proto.v0 import statistics_pb2
from tensorflow_transform.tf_metadata import schema_utils
import tensorflow_data_validation as tfdv

print("Schema from SchemaGen:")
_list = store.get_artifacts_by_type("Schema")
print(f'Schema count={len(_list)}')
_list = sorted(_list, key=lambda x: x.create_time_since_epoch, reverse=True)
for artifact in _list:
    if "SchemaGen" in artifact.uri:
        artifact_uri = artifact.uri
        break
assert(artifact_uri is not None)
file_path = os.path.join(artifact_uri, "schema.pbtxt")
schema = tfdv.load_schema_text(file_path)
#tfdv.visualize_artifacts(schema)
tfdv.display_schema(schema=schema)

print("ExampleStatistcs from StatisticsGen:")
_list = store.get_artifacts_by_type("ExampleStatistics")
print(f'ExampleStatistics count={len(_list)}')
#print(f'ExampleStatistics ={_list}')
_list = sorted(_list, key=lambda x: x.create_time_since_epoch, reverse=True)
for artifact in _list:
    if "StatisticsGen" in artifact.uri:
        artifact_uri = artifact.uri
        break
assert(artifact_uri is not None)
file_paths = [os.path.join(artifact_uri, name, "FeatureStats.pb") 
  for name in os.listdir(artifact_uri)]
stats_dict = {}
for file_path in file_paths:
    if "test" in file_path:
        #plotting train and eval
        continue
    try:
        with open(file_path, 'rb') as f:
            serialized_stats = f.read()
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")    
    stats = statistics_pb2.DatasetFeatureStatisticsList()
    stats.ParseFromString(serialized_stats)
    if "train" in file_path:
        stats_dict['train'] = stats
    else:
        stats_dict['eval'] = stats
tfdv.visualize_statistics(lhs_statistics=stats_dict['eval'],
    rhs_statistics=stats_dict['train'],
    lhs_name='EVAL_DATASET', rhs_name='TRAIN_DATASET')

print("ExampleAnomalies from ExampleValidator:")
_list = store.get_artifacts_by_type("ExampleAnomalies")
print(f'ExampleAnomalies count={len(_list)}')
_list = sorted(_list, key=lambda x: x.create_time_since_epoch, reverse=True)
for artifact in _list:
    if "ExampleValidator" in artifact.uri:
        artifact_uri = artifact.uri
        break
assert(artifact_uri is not None)
file_paths = [os.path.join(artifact_uri, name, "SchemaDiff.pb") 
  for name in os.listdir(artifact_uri)]
for file_path in file_paths:
    try:
        with open(file_path, 'rb') as f:
            serialized_anomalies = f.read()
    except FileNotFoundError:
        print(f"Error: Anomaly file not found at {file_path}")
    anomalies = anomalies_pb2.Anomalies()
    anomalies.ParseFromString(serialized_anomalies)
    tfdv.display_anomalies(anomalies)

print("ExampleStatistics from pre-transform stats of Transform:")
_list = store.get_artifacts_by_type("ExampleStatistics")
#print(f'ExampleStatistics={_list}')
_list = sorted(_list, key=lambda x: x.create_time_since_epoch, reverse=True)
for artifact in _list:
    if "pre_transform_stats" in artifact.uri:
        artifact_uri = artifact.uri
        break
assert(artifact_uri is not None)
file_paths = [os.path.join(artifact_uri, name) 
  for name in os.listdir(artifact_uri)]
for file_path in file_paths:
    try:
        with open(file_path, 'rb') as f:
            serialized_stats = f.read()
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")    
    stats = statistics_pb2.DatasetFeatureStatisticsList()
    stats.ParseFromString(serialized_stats)
    tfdv.visualize_statistics(stats)
    

Schema from SchemaGen:
Schema count=3


,Type,Presence,Valency,Domain
Feature name,,,,
'genres',BYTES,required,,-
'age',INT,required,,-
'gender',STRING,required,,'gender'
'movie_id',INT,required,,-
'occupation',INT,required,,-
'rating',INT,required,,-
'timestamp',INT,required,,-
'user_id',INT,required,,-


,Values
Domain,
'gender',"'F', 'M'"


ExampleStatistcs from StatisticsGen:
ExampleStatistics count=3


ExampleAnomalies from ExampleValidator:
ExampleAnomalies count=2


ExampleStatistics from pre-transform stats of Transform:


### using TFDV to look at the transformed data

In [21]:

print("Schema from post-Transform:")
_list = store.get_artifacts_by_type("Schema")
print(f'Schema count={len(_list)}')
_list = sorted(_list, key=lambda x: x.create_time_since_epoch, reverse=True)
for artifact in _list:
    if "post_transform_schema" in artifact.uri:
        artifact_uri = artifact.uri
        break
assert(artifact_uri is not None)
file_path = os.path.join(artifact_uri, "schema.pbtxt")
schema = tfdv.load_schema_text(file_path)
#tfdv.visualize_artifacts(schema)
tfdv.display_schema(schema=schema)

print("ExampleStatistics from post-transform stats of Transform:")
_list = store.get_artifacts_by_type("ExampleStatistics")
_list = sorted(_list, key=lambda x: x.create_time_since_epoch, reverse=True)
for artifact in _list:
    if "post_transform_stats" in artifact.uri:
        artifact_uri = artifact.uri
        break
assert(artifact_uri is not None)
file_paths = [os.path.join(artifact_uri, name) 
  for name in os.listdir(artifact_uri)]
for file_path in file_paths:
    try:
        with open(file_path, 'rb') as f:
            serialized_stats = f.read()
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")    
    stats = statistics_pb2.DatasetFeatureStatisticsList()
    stats.ParseFromString(serialized_stats)
    tfdv.visualize_statistics(stats)
    
print("ExampleAnomalies from post-transform of Transform:")
_list = store.get_artifacts_by_type("ExampleAnomalies")
_list = sorted(_list, key=lambda x: x.create_time_since_epoch, reverse=True)
for artifact in _list:
    if "post_transform_anomalies" in artifact.uri:
        artifact_uri = artifact.uri
        break
assert(artifact_uri is not None)
file_paths = [os.path.join(artifact_uri, name) 
  for name in os.listdir(artifact_uri)]
for file_path in file_paths:
    anomalies = anomalies_pb2.Anomalies()
    try:
        with open(file_path, 'rb') as f:
            serialized_anomalies = f.read()
    except FileNotFoundError:
        print(f"Error: Anomaly file not found at {file_path}")
    anomalies = anomalies_pb2.Anomalies()
    anomalies.ParseFromString(serialized_anomalies)
    tfdv.display_anomalies(anomalies)


Schema from post-Transform:
Schema count=3


,Type,Presence,Valency,Domain
Feature name,,,,
'age',FLOAT,required,,-
'gender',FLOAT,required,,-
'genres',FLOAT,required,,-
'hr',FLOAT,required,,-
'hr_wk',FLOAT,required,,-
'month',FLOAT,required,,-
'movie_id',FLOAT,required,,-
'occupation',FLOAT,required,,-
'rating',FLOAT,required,,-


ExampleStatistics from post-transform stats of Transform:


ExampleAnomalies from post-transform of Transform:


## Save the data schema with version control

In [20]:
import shutil

_list = store.get_artifacts_by_type("Schema")
_list = sorted(_list, key=lambda x: x.create_time_since_epoch, reverse=True)
for artifact in _list:
    if "pre_transform_schema" in artifact.uri:
        artifact_uri = artifact.uri
        break
assert(artifact_uri is not None)
pre_transform_file_path = [os.path.join(artifact_uri, name) for name in os.listdir(artifact_uri)][0]

for artifact in _list:
    if "post_transform_schema" in artifact.uri:
        artifact_uri = artifact.uri
        break
assert(artifact_uri is not None)
post_transform_file_path = [os.path.join(artifact_uri, name) for name in os.listdir(artifact_uri)][0]

pre_dir = os.path.join(get_project_dir(), "src/main/resources", "pre_transform")
post_dir = os.path.join(get_project_dir(), "src/main/resources", "post_transform")
os.makedirs(pre_dir, exist_ok=True)
os.makedirs(post_dir, exist_ok=True)

pre_schema_path = os.path.join(pre_dir, "schema.pbtxt")
post_schema_path = os.path.join(post_dir, "schema.pbtxt")

shutil.copyfile(pre_transform_file_path, pre_schema_path)
shutil.copyfile(post_transform_file_path, post_schema_path)


'/home/nichole/projects/github/recommender_systems/src/main/resources/post_transform/schema.pbtxt'

## Run baseline model pipeline with full dataset

pipeline_factory = PipelineComponentsFactory(
  infiles_dict_ser=infiles_dict_ser, output_config_ser=output_config_ser,
  transform_dir=tr_dir, user_id_max=user_id_max, movie_id_max=movie_id_max,
  n_genres=n_genres, n_age_groups=n_age_groups, min_eval_size=MIN_EVAL_SIZE,
  serving_model_dir=serving_model_dir,
)

beam_pipeline_args = [
  '--direct_running_mode=multi_processing',
  '--direct_num_workers=0']

baseline_components = pipeline_factory.build_components(PIPELINE_TYPE.BASELINE,
  run_example_diff=False, pre_transform_schema=pre_schema_path,
  post_transform_schema=post_schema_path)
    
# create baseline model
my_pipeline = tfx.dsl.Pipeline(
  pipeline_name=PIPELINE_NAME,
  pipeline_root=PIPELINE_ROOT,
  components=baseline_components,
  enable_cache=ENABLE_CACHE,
  metadata_connection_config=metadata_connection_config,
  beam_pipeline_args=beam_pipeline_args,
)

tfx.orchestration.LocalDagRunner().run(my_pipeline)


artifact_types = store.get_artifact_types()
logging.debug(f"MLMD store artifact_types={artifact_types}")
artifacts = store.get_artifacts()
logging.debug(f"MLMD store artifacts={artifacts}")

components = pipeline_factory.build_components(PIPELINE_TYPE.BASELINE,
  run_example_diff=False, pre_transform_schema=pre_schema_path,
  post_transform_schema=post_schema_path)
# simulate experimentation of one model family
my_pipeline = tfx.dsl.Pipeline(
  pipeline_name=PIPELINE_NAME,
  pipeline_root=PIPELINE_ROOT,
  components=components,
  enable_cache=ENABLE_CACHE,
  metadata_connection_config=metadata_connection_config,
  beam_pipeline_args=beam_pipeline_args,
)

tfx.orchestration.LocalDagRunner().run(my_pipeline)

#TODO: save schema.pbtxt to repository for version control


artifact_types = store.get_artifact_types()
print(f"MLMD store artifact_types={artifact_types}")
artifacts = store.get_artifacts()
print(f"MLMD store artifacts={artifacts}")

executions = store.get_executions()
logging.debug(f"MLMD store executions={executions}")

# executions has custom_properties.key: "infiles_dict_ser"
#    and custom_properties.key: "output_config_ser"
artifact_count = len(artifacts)
execution_count = len(executions)
